In [1]:
from rdkit import Chem
import pyaniasetools as pya

from ase_interface import ANIENS,ensemblemolecule
import hdnntools as hdt
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

np.set_printoptions(threshold=1000000000,suppress=True)

van Der Waals correction will be unavailable. Please install ased3


In [2]:
ntdir = '/nh/nest/u/jsmith/scratch/Gits/ANI-Networks/networks/al_networks/ANI-AL-0808.0303.0400/'
cns = ntdir + 'train0/rHCNOSFCl-4.6A_16-3.1A_a4-8.params'
sae = ntdir + 'train0/sae_wb97x-631gd.dat'
nnf = ntdir + 'train'

#ntdir = '/nh/nest/u/jsmith/scratch/Research/ccsd_extrapolation/new_ccsd_data/ccsd_train/'
#cns = ntdir + 'rHCNO-4.6A_16-3.1A_a4-8.params'
#sae = ntdir + 'sae_linfit.dat'
#nnf = ntdir + 'train'

datai = "/nh/nest/u/jsmith/Research/Data_Oles/drugbank_3d_1564.sdf"
datao = "/nh/nest/u/jsmith/Research/Data_Oles/ani_data.sdf"

Nn = 5

In [3]:
# Set ANI calculator
aec = pya.anienscomputetool(cns, sae, nnf, Nn, 0)

In [4]:
suppl = Chem.SDMolSupplier(datai, removeHs=False)

In [5]:
writer = Chem.SDWriter(datao)
for i,mol in enumerate(suppl):
    X, S = pya.__convert_rdkitmol_to_nparr__(mol)
    if (set(S).issubset(['C', 'H', 'N', 'O', 'S', 'F', 'Cl'])):
        print("Molecule:",i,mol.GetNumAtoms(),set(S))
        opt = aec.optimize_rdkit_molecule(mol,-1)
        E,sig = aec.energy_rdkit_conformers(mol,[0])
        
        mol.SetProp("ANIenergy",str(E))
        mol.SetProp("ANIstddev",str(sig))
        
        activations = dict({"layer0":[],"layer1":[],"layer2":[],})
        for i in range(3):
            for j in range(mol.GetNumAtoms()):
                activations["layer"+str(i)].append(aec.ens.ncl[0].activations(j,i,0))
            activations["layer"+str(i)] = np.vstack(activations["layer"+str(i)])
            
        for k in activations.keys():
            mol.SetProp(k,np.array2string(activations[k]))
        
        writer.write(mol)

Molecule: 0 55 {'O', 'C', 'N', 'H'}
Molecule: 1 20 {'O', 'C', 'N', 'H'}
Molecule: 2 20 {'O', 'C', 'N', 'H'}
Molecule: 3 10 {'O', 'C', 'H'}
Molecule: 4 23 {'O', 'C', 'N', 'H'}
Molecule: 5 32 {'C', 'N', 'H', 'S', 'O'}
Molecule: 6 24 {'O', 'C', 'N', 'H'}
Molecule: 7 26 {'O', 'C', 'N', 'H'}
Molecule: 8 20 {'O', 'C', 'H'}
Molecule: 9 40 {'C', 'N', 'H'}
Molecule: 10 16 {'O', 'C', 'N', 'H'}
Molecule: 11 21 {'O', 'C', 'N', 'H'}
Molecule: 12 20 {'O', 'C', 'N', 'H'}
Molecule: 14 50 {'O', 'C', 'H'}
Molecule: 15 14 {'O', 'C', 'N', 'H'}
Molecule: 16 20 {'C', 'N', 'H', 'S', 'O'}
Molecule: 17 24 {'O', 'C', 'N', 'H'}
Molecule: 18 74 {'O', 'C', 'H'}
Molecule: 19 98 {'O', 'C', 'H'}
Molecule: 20 26 {'C', 'N', 'H', 'S', 'O'}
Molecule: 21 14 {'O', 'C', 'H'}
Molecule: 22 47 {'O', 'C', 'N', 'H'}
Molecule: 23 30 {'O', 'C', 'N', 'H'}
Molecule: 24 37 {'C', 'N', 'H', 'S', 'O'}
Molecule: 26 10 {'O', 'C', 'N', 'H'}
Molecule: 27 73 {'O', 'C', 'H'}
Molecule: 28 21 {'O', 'C', 'N', 'H'}
Molecule: 29 18 {'O', 'C', 'N',

KeyboardInterrupt: 